In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time

In [2]:
def get_info(hotel_name):
    url = hotel_name.find_element_by_tag_name('a').get_attribute('href')
    
    driver.get(url)

In [3]:
browser = webdriver.Chrome('./chromedriver')

In [4]:
# driver.get('https://me.cleartrip.com/hotels/united-states/miami/')
# time.sleep(10)

In [5]:
num_pages = 38

In [6]:
# soup = BeautifulSoup(driver.page_source, 'html.parser')

In [7]:
# type(soup)

In [8]:
# page_link_elems = driver.find_element_by_class_name('pagination').find_elements_by_tag_name('a')

In [9]:
# page_hrefs = [x.get_attribute('href') for x in page_links]

NameError: name 'page_links' is not defined

#### hotels in the first page, ie, on https://me.cleartrip.com/hotels/united-states/miami/

In [ ]:
hotel_name_elems = driver.find_elements_by_class_name('hotels-name')

In [ ]:
hotel_